In [ ]:
import math
import numpy as np
import scipy.sparse
import sys
import os
import matplotlib.pyplot as plt
import networkx as nx
import networkx.algorithms.approximation as nx_app
import math
np.set_printoptions(threshold=sys.maxsize)

current = os.path.dirname(os.path.realpath('__file__'))
parent = os.path.dirname(current)
parent = os.path.dirname(parent)
parent = os.path.dirname(parent)
parent = os.path.dirname(parent)


sys.path.append(parent)
import qhea.solver
import qhea.parameters
import qhea.problem
import benchmark
import tsp_qubo
import plotter
import annealing

In [ ]:
tsp = np.array(tsp_qubo.from_xml.get_problem('kroD100.xml'))
n_nodes = len(tsp)
penalty = tsp.max()

In [ ]:
parameters = qhea.parameters.Parameters(
    optimiziation_rate=0.005,
    n_individuals=100,
    n_generations=1000,
    selection_pressure=2,
    bias=0.5
)
qubo_tsp = qhea.problem.TSPQUBO(tsp, 100, penalty)
solver = qhea.solver.Qhea(qubo_tsp, parameters, annealing.d_wave.anneal)
res = solver.optimize()
qhea_result = res.all_time_fittest[1]
qhea_objective = res.all_time_fittest[0]
print(np.sum(np.reshape(qhea_result, (n_nodes,n_nodes)),axis=0))
print(np.sum(np.reshape(qhea_result, (n_nodes,n_nodes)),axis=1))



In [ ]:
problem = qubo_tsp._generate_q_mat(penalty)
sim_anneal_result, sim_anneal_objective = annealing.d_wave.anneal(problem)

In [ ]:
tabu_result, tabu_objective = annealing.d_wave.tabu(problem)

In [ ]:
def resolve_to_permutation(x, shape):
    b = np.reshape(x, shape)

    permutation = []

    for i in range(len(b)):
        permutation.append(np.where(b[i] == 1)[0][0])

    permutation.append(permutation[0])
    return permutation

def resolve_to_distance(x, d_mat):
    t_mat = np.reshape(x, (n_nodes, n_nodes))
    result = 0
    for t in range(n_nodes):
        result += t_mat[t] @ d_mat @ t_mat[(t + 1) % n_nodes]

    return result


def get_edge_list(x, shape):
    permutation = resolve_to_permutation(x,shape)
    edge_list = list(nx.utils.pairwise(permutation))
    return edge_list


In [ ]:
print(f'Qhea result: {qhea_objective} ')
print(f'Tabu Result: {tabu_objective} ')
print(f'Sim result: {sim_anneal_objective} ')

print(f'Qhea distance: {resolve_to_distance(qhea_result,tsp)}')
print(f'Tabu distance: {resolve_to_distance(tabu_result,tsp)}')
print(f'Sim distance: {resolve_to_distance(sim_anneal_result,tsp)}')

print(resolve_to_permutation(qhea_result, (100,100)))